<a href="https://colab.research.google.com/github/MarioROT/IHLT-MAI/blob/main/Session5_MarioRosas_AlamLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 5 (Lexical Semmantics) - IHLT

**Students:**
- Mario Rosas
- Alam Lopez

**Lab Professor:** Salvador Medina Herrera

## Paraphrases Template

In [1]:
import nltk
import pandas as pd
pd.set_option("display.precision", 3)
nltk.download('treebank')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('omw-1.4')
nltk.download('wordnet_ic')
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


## TODO

 **Given the following (lemma, category) pairs:**   \

```
('the','DT'),('man','NN'),('swim','VB'),('with','PR'),('a','DT'),('girl','NN'),('and','CC'),
('a','DT'),('boy','NN'),('whilst','PR'),('the','DT'),('woman','NN'),('walk','VB')
```

1. For each pair, when possible, print their most frequent WordNet synset

2. For each pair of words, when possible, print their corresponding least common subsumer (LCS) and their similarity value, using the following functions:

  - Path Similarity
  - Leacock-Chodorow Similarity
  - Wu-Palmer Similarity
  - Lin Similarity

3. Normalize similarity values when necessary. What similarity seems better?

In [2]:
Lemmas= [('the','DT'),('man','NN'),('swim','VB'),('with','PR'),('a','DT'),('girl','NN'),('and','CC'),
('a','DT'),('boy','NN'),('whilst','PR'),('the','DT'),('woman','NN'),('walk','VB')]

In [3]:
#We apply a cleaning process to remove some possible duplicate values
Clean_pairs = list(set(Lemmas))
print(len(Lemmas)-len(Clean_pairs), 'lemma-category values are duplicates')

2 lemma-category values are duplicates


In [4]:
# Function to compute the most frequent synset
def most_frequent_synset(word, pos):
  syns = wn.synsets(word, pos)
  m = 0
  res = None
  for s in syns:
      for l in s.lemmas():
          if l.count() > m:
            res = l
  return res.synset()

In [5]:
#Ex. 1 With the corresponding pairs cleaned and with the correct category id
# we can now review the most frequen synset

d = {'NN': 'n', 'NNS': 'n',
       'JJ': 'a', 'JJR': 'a', 'JJS': 'a',
       'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v',
       'RB': 'r', 'RBR': 'r', 'RBS': 'r'}

def most_frequent_for_all(Lemmas):
  results={'accepted':[], 'discarded':[]}
  for pair in range(len(Lemmas)):

    if Lemmas[pair][1] in d.keys():

      if len(wn.synsets(Lemmas[pair][0],d[Lemmas[pair][1]]))==0:
         results['discarded'].append(Lemmas[pair])
      else:
        common= most_frequent_synset(Lemmas[pair][0],d[Lemmas[pair][1]])
        tupple=(Lemmas[pair][0],common)
        results['accepted'].append(tupple)

    else:
      results['discarded'].append(Lemmas[pair])

  return results

In [6]:
Lemmas_syn=most_frequent_for_all(Clean_pairs)
pd.DataFrame.from_dict(Lemmas_syn, orient='index').T

,accepted,discarded
0,"(woman, Synset('woman.n.02'))","(a, DT)"
1,"(walk, Synset('walk.v.05'))","(and, CC)"
2,"(girl, Synset('girl.n.05'))","(the, DT)"
3,"(boy, Synset('boy.n.04'))","(whilst, PR)"
4,"(man, Synset('valet.n.01'))","(with, PR)"
5,"(swim, Synset('float.v.02'))",None


In [7]:
#Now, we put all the lemma-category in all the possible pair combinations into the list "Pairs"
#Since there are 11 pairs, we should get 55 elements
Pairs = [(a, b) for idx, a in enumerate(Lemmas_syn['accepted']) for b in Lemmas_syn['accepted'][idx + 1:]]

In [8]:
def process_name(p1,p2):
  return p1.ljust(6) + '--' + p2.rjust(6)

In [9]:
# Computing the similiatries and nomalizing the ones in a different scale

from nltk.corpus.reader.wordnet import lch_similarity
similarity_results = ['lcs','path','lch','wup','lin']
print_pairs = {}
for pair in Pairs:
  pair_name = process_name(pair[0][0],pair[1][0])
  print_pairs[pair_name] = []
  try:
    print_pairs[pair_name].append(pair[0][1].lowest_common_hypernyms(pair[1][1])[0].name())
  except:
    print_pairs[pair_name].append('')
  print_pairs[pair_name].append(pair[0][1].path_similarity(pair[1][1]))
  try:
    print_pairs[pair_name].append(pair[0][1].lch_similarity(pair[1][1]))
  except:
    print_pairs[pair_name].append(0)
  print_pairs[pair_name].append(pair[0][1].wup_similarity(pair[1][1]))
  try:
    print_pairs[pair_name].append(pair[0][1].lin_similarity(pair[1][1],brown_ic))
  except:
    print_pairs[pair_name].append(0)

res = pd.DataFrame(print_pairs).T
res.columns = similarity_results
res['lch'] = (res['lch'] - res['lch'].min())/(res['lch'].max()-res['lch'].min())
res

,lcs,path,lch,wup,lin
woman -- walk,,0.091,0.0,0.167,0
woman -- girl,female.n.02,0.25,1.0,0.632,0.7
woman -- boy,person.n.01,0.167,0.82,0.632,0.276
woman -- man,person.n.01,0.143,0.751,0.6,0.305
woman -- swim,,0.1,0.0,0.182,0
walk -- girl,,0.083,0.0,0.154,0
walk -- boy,,0.083,0.0,0.154,0
walk -- man,,0.077,0.0,0.143,0
walk -- swim,,0.167,0.651,0.286,0.0
girl -- boy,adult.n.01,0.167,0.82,0.6,0.576


# Conclusion

First, we need to take into account that each of the similarities are computed in a different way, and therefore each of it, may have some specific considerations.

For example, it may seem that the Leacock Chordorow is the one that has the higher similarities, however since we are normalizing the values by considering all the pair of words we are using, it has a relative value and more sensitivity to the range of the values we get in the whole analysis.

In the case of the lin similarity is important to notice that in order to be computed, the PoS of both of the words should be the same.

As we see in the table, the type of similarity that keeps the most consistency across the pair of words, by giving a low variance value is the Wu-Palmer Similarity. So from this point of view of consistency, Wu-palmer could be provide useful information about the level of relationships between the analyzed words pairs.
